In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [4]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


In [4]:
# data_augmentation = keras.Sequential(
# [
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.2),
#     layers.RandomContrast(0.2),
#     layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
# ] 
# )

In [6]:
inputs = keras.Input(shape=(100, 100, 3))
x = layers.Conv2D(16, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(22, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 22)        6358

In [7]:
callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=7
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_7.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

In [8]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 10:16:55.946766: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 10:16:55.946851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


104/105 [============================>.] - ETA: 0s - loss: 2.7020 - accuracy: 0.1986

2022-08-01 10:16:59.187467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 4s 29ms/step - loss: 2.6930 - accuracy: 0.1982 - val_loss: 1.7365 - val_accuracy: 0.2639
Epoch 2/40
105/105 [==============================] - 3s 24ms/step - loss: 1.8195 - accuracy: 0.2577 - val_loss: 1.6924 - val_accuracy: 0.3597
Epoch 3/40
105/105 [==============================] - 2s 23ms/step - loss: 1.7551 - accuracy: 0.3036 - val_loss: 1.6080 - val_accuracy: 0.3903
Epoch 4/40
105/105 [==============================] - 2s 23ms/step - loss: 1.7170 - accuracy: 0.3500 - val_loss: 1.5092 - val_accuracy: 0.5014
Epoch 5/40
105/105 [==============================] - 2s 23ms/step - loss: 1.5930 - accuracy: 0.3821 - val_loss: 1.3469 - val_accuracy: 0.5361
Epoch 6/40
105/105 [==============================] - 2s 23ms/step - loss: 1.4594 - accuracy: 0.4324 - val_loss: 1.1664 - val_accuracy: 0.5958
Epoch 7/40
105/105 [==============================] - 2s 23ms/step - loss: 1.3165 - accuracy: 0.4943 - val_loss: 1.3127 - val_accuracy: 0.4417
Epoch 8/40

In [8]:
model.evaluate(test)

23/23 [==============================] - 0s 12ms/step - loss: 0.4365 - accuracy: 0.9069


[0.4364646077156067, 0.9069444537162781]

In [9]:
# keras.models.save_model(model, "./models/zenodo_3.h5")

load_model = keras.models.load_model("models/zenodo_7.h5")
load_model.evaluate(test)

 9/23 [==========>...................] - ETA: 0s - loss: 0.2891 - accuracy: 0.9132

2022-08-01 10:09:16.565756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 14ms/step - loss: 0.3014 - accuracy: 0.9125


[0.3013952672481537, 0.9125000238418579]